<a href="https://colab.research.google.com/github/MarcoMartini75/dati/blob/master/test_logreg_gridserachcv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone --recursive https://github.com/MarcoMartini75/dati

Cloning into 'dati'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 12 (delta 4), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [0]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import itertools

from wordcloud import WordCloud
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from textblob import TextBlob # for spelling correction
#from nltk.stem import SnowballStemmer ==> à voir
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split


# Models
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import ensemble
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB



from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import f1_score

from time import time

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

#scripts perso:
#import preprocess as pre

In [3]:
from google.colab import files
files.upload()

Saving preprocess.py to preprocess.py


{'preprocess.py': b'###########################\r\n# PREPROCESSING FUNCTIONS \r\n# @Author : Edithe Selwa\r\n# @Date : Jan 2019\r\n# @Projet e-commerce\r\n# @Datascientest\r\n###########################\r\n\r\n####################\r\n#    LIBRAIRIES    #\r\n####################\r\nimport numpy as np\r\nimport pandas as pd\r\nimport seaborn as sns\r\nimport re\r\nimport itertools\r\n\r\nfrom wordcloud import WordCloud\r\nfrom nltk.corpus import stopwords\r\nfrom nltk.tokenize import TweetTokenizer\r\nfrom nltk.tokenize import word_tokenize\r\nfrom nltk.tokenize import RegexpTokenizer\r\nfrom nltk.stem import PorterStemmer\r\nfrom textblob import TextBlob \r\n\r\n\r\n########################\r\n# TEXT-PREPROCESSING   #\r\n########################\r\n\r\n##   STOP WORDS   ##\r\ndef clean_stop_words():\r\n    stop_words = set(stopwords.words(\'english\'))\r\n    #print(stop_words)\r\n    # ==> les stop words contiennent des mots qui pourraient \xc3\xaatre utiles pour l\'analyse des sentime

In [0]:
import preprocess as pre

In [5]:
df = pd.read_csv("/content/dati/Womens Clothing E-Commerce Reviews.csv", index_col = 0)
df.head(10)

,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses
5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
6,858,39,Cagrcoal shimmer fun,I aded this in my basket at hte last mintue to...,5,1,1,General Petite,Tops,Knits
7,858,39,"Shimmer, surprisingly goes with lots","I ordered this in carbon for store pick up, an...",4,1,4,General Petite,Tops,Knits
8,1077,24,Flattering,I love this dress. i usually get an xs but it ...,5,1,0,General,Dresses,Dresses
9,1077,34,Such a fun dress!,"I'm 5""5' and 125 lbs. i ordered the s petite t...",5,1,0,General,Dresses,Dresses


In [6]:
df1 = df.dropna(subset=["Title"])

# Attention : on ne fait pas un bag of words. On veut preprocesser sur chaque ligne.
# Text cleaning line by line:
list_line=[]
for line in df1["Title"]:    
    list_line.append(pre.text_cleaning(line))
print(list_line[:10])

X_train, X_test, y_train, y_test = train_test_split(list_line, df1["Recommended IND"], test_size=0.2, random_state = 123)

['some major design flaws', 'my favorite buy ! ', 'flattering shirt', 'not for the very petite', 'cagrcoal shimmer fun', 'shimmer surprisingly goes with lots', 'flattering', 'such a fun dress ! ', 'dress looks like it made of cheap material', 'perfect ! ! ! ']


In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
#Update stopwords
stopw=pre.clean_stop_words()
#normalisation TfIDF
vect_title = TfidfVectorizer(stop_words=stopw, tokenizer=pre.prePro)
X_train = vect_title.fit_transform(X_train)
X_test = vect_title.transform(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'ourselv', 'themselv', 'thi', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


## Standard calculation

In [10]:
    clf = LogisticRegression(C = 1.0)
    t0 = time()
    clf.fit(X_train.todense(), y_train)
    y_pred = clf.predict(X_test.todense())

    t1 = time() - t0
    print("Réalisé en {} secondes".format(round(t1,3)))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Réalisé en 0.532 secondes


In [11]:
print( classification_report(y_test, y_pred) )

cm_lr = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
print(cm_lr)

              precision    recall  f1-score   support

           0       0.76      0.50      0.61       710
           1       0.90      0.97      0.93      3226

   micro avg       0.88      0.88      0.88      3936
   macro avg       0.83      0.73      0.77      3936
weighted avg       0.87      0.88      0.87      3936

Classe prédite    0     1
Classe réelle            
0               358   352
1               112  3114


## GridSerchCV (scoring = accuracy)

In [13]:
from sklearn.model_selection import GridSearchCV

t0 = time()

grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(X_train.todense(),y_train)

t1 = time() - t0
print("Réalisé en {} secondes".format(round(t1,3)))

print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

Réalisé en 55.28 secondes
tuned hpyerparameters :(best parameters)  {'C': 10.0, 'penalty': 'l2'}
accuracy : 0.8818932655654383


In [0]:
y_pred = logreg_cv.predict(X_test.todense())

In [15]:
print( classification_report(y_test, y_pred) )

cm_lrcv = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
print(cm_lrcv)

              precision    recall  f1-score   support

           0       0.74      0.60      0.66       710
           1       0.92      0.95      0.93      3226

   micro avg       0.89      0.89      0.89      3936
   macro avg       0.83      0.78      0.80      3936
weighted avg       0.88      0.89      0.89      3936

Classe prédite    0     1
Classe réelle            
0               429   281
1               154  3072


Comme ca la classe 0 est predite mieux

## GridSearchCV (scoring=ROC AUC)

In [16]:
t0 = time()

grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg=LogisticRegression()
logreg_cv_roc=GridSearchCV(logreg,grid,scoring='roc_auc',cv=10)
logreg_cv_roc.fit(X_train.todense(),y_train)

t1 = time() - t0
print("Réalisé en {} secondes".format(round(t1,3)))

print("tuned hpyerparameters :(best parameters) ",logreg_cv_roc.best_params_)
print("accuracy :",logreg_cv_roc.best_score_)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

Réalisé en 63.608 secondes
tuned hpyerparameters :(best parameters)  {'C': 1.0, 'penalty': 'l2'}
accuracy : 0.9238130680300363


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [0]:
y_pred = logreg_cv_roc.predict(X_test.todense())

In [18]:
print( classification_report(y_test, y_pred) )

cm_lrcv_roc = pd.crosstab(y_test, y_pred, rownames=['Classe réelle'], colnames=['Classe prédite'])
print(cm_lrcv_roc)

              precision    recall  f1-score   support

           0       0.76      0.50      0.61       710
           1       0.90      0.97      0.93      3226

   micro avg       0.88      0.88      0.88      3936
   macro avg       0.83      0.73      0.77      3936
weighted avg       0.87      0.88      0.87      3936

Classe prédite    0     1
Classe réelle            
0               358   352
1               112  3114


On retombe sur le premier résultat etant donné que les paramètres choisis par CV sont les memes que dans le premier calcul. Personellement je m'attendais à avoir une prediction meilleure pour la classe 0.